## 원본 라벨링 데이터 로드

In [1]:
from collections import Counter
import json
import os
import re
import json

In [2]:
def read_json_file(fp):
    with open(fp, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

def load_json_files(dir):
    json_data_list = []

    for filename in os.listdir(dir):
        if filename.endswith('.json'):
            fp = os.path.join(dir, filename)
            json_data = read_json_file(fp)
            json_data_list.extend(json_data)
    
    return json_data_list

In [3]:
files_path = './absa_data/labeling_data'
all_json_data = load_json_files(files_path)

len(all_json_data)

49996

In [4]:
all_json_data[0]

{'Index': '215137',
 'RawText': '유통기한도 넉넉하고  구성도 많아서 선물 하기 좋네요.   만족합니다.',
 'Source': '쇼핑몰',
 'Domain': '화장품',
 'MainCategory': '스킨케어',
 'ProductName': 'OO 프리미엄 GE라인 에스테틱 패키지 13종',
 'ReviewScore': '100',
 'Syllable': '39',
 'Word': '8',
 'RDate': '20210920',
 'GeneralPolarity': '1',
 'Aspects': [{'Aspect': '유통기한',
   'SentimentText': '유통기한도 넉넉하고',
   'SentimentWord': '2',
   'SentimentPolarity': '1'},
  {'Aspect': '제품구성',
   'SentimentText': '구성도 많아서 선물 하기 좋네요.',
   'SentimentWord': '5',
   'SentimentPolarity': '1'}]}

## 데이터 전처리

In [5]:
# {"MainCategory": "남성화장품", "raw_text": "", "annotation": [["가격", "1"], ["유통기한", "-1"]]}
converted_data = []

for review in all_json_data:
    raw_text = re.sub(r"[^가-힣| |]+", " ", review['RawText']) # 한글만 남기기
    raw_text = re.sub(' +', ' ', raw_text).rstrip() # 공백 연속 제거, 맨 끝 공백 삭제
    aspects = [[aspect['Aspect'], aspect['SentimentPolarity']] for aspect in review.get('Aspects')]

    converted_review = {"MainCategory": review['MainCategory'], "raw_text": raw_text, "annotation": aspects}
    converted_data.append(converted_review)

In [6]:
len(converted_data)

49996

In [7]:
converted_data[-2:-1]

[{'MainCategory': '남성화장품',
  'raw_text': '간편하게 하나만 발라도 되어서 신랑이 좋아하네요 향기도 끝내줍니다 제가 너무 좋아하는 향이라 앞으로도 계속 사용할 것 같습니다',
  'annotation': [['편의성/활용성', '1'], ['향', '1'], ['향', '1']]}]

In [8]:
### 겹치는 aspect 랜덤으로 하나 남기고 제거 - set 사용
for review in converted_data:
    unique_annotation = {}
    for k, v in review.get('annotation', []):
        if k not in unique_annotation:
            unique_annotation[k] = v

    review['annotation'] = list(unique_annotation.items())

In [9]:
converted_data[-2:-1]

[{'MainCategory': '남성화장품',
  'raw_text': '간편하게 하나만 발라도 되어서 신랑이 좋아하네요 향기도 끝내줍니다 제가 너무 좋아하는 향이라 앞으로도 계속 사용할 것 같습니다',
  'annotation': [('편의성/활용성', '1'), ('향', '1')]}]

## 중복 제거 후 개수 확인

In [10]:
# MainCategory별로 너무 적은 속성은 확인
category_aspect_counts = {}

for entry in converted_data:
    main_category = entry['MainCategory']
    aspects = entry['annotation']

    if main_category not in category_aspect_counts.keys():
        category_aspect_counts[main_category] = Counter()

    aspect_texts = [aspect[0] for aspect in aspects]
    category_aspect_counts[main_category].update(aspect_texts)

# 결과 출력
for main_category, aspect_counts in category_aspect_counts.items():
    print(f"MainCategory: {main_category}")

    # Aspect를 오름차순으로 정렬
    sorted_aspects = sorted(aspect_counts.items(), key=lambda x: x[0])
    
    for aspect, count in sorted_aspects:
        print(f"  {aspect}: {count}")
    print("\n")

MainCategory: 스킨케어
  가격: 4160
  기능/효과: 8011
  디자인: 93
  밀착력/접착력: 70
  발림성: 2757
  보습력/수분감: 7566
  사용감: 526
  색상: 142
  성분: 686
  용기: 556
  용량: 1940
  유통기한: 588
  윤기/피부(톤): 2888
  자극성: 2277
  제품구성: 2342
  제형: 1651
  지속력: 967
  커버력: 67
  탄력: 853
  편의성/활용성: 2807
  품질: 653
  피부타입: 1241
  향: 2641
  흡수력: 4088


MainCategory: 헤어/바디케어
  가격: 4274
  거품력: 826
  그립감: 8
  기능/효과: 3274
  두피보호: 290
  디자인: 98
  머릿결관리: 2831
  밀착력/접착력: 64
  발림성: 1053
  발색력: 757
  보습력/수분감: 2825
  분사력: 139
  사용감: 370
  색상: 369
  성분: 603
  세정력: 1058
  세팅력/고정력: 870
  스타일링효과: 985
  염색력: 1906
  용기: 691
  용량/사이즈: 1743
  유통기한: 518
  윤기/피부(톤): 183
  이염: 314
  자극성: 1741
  제품구성: 1194
  제형: 1233
  지속력/유지력: 1504
  청량감/쿨링감: 1110
  클렌징/제거력: 339
  탈모개선: 935
  편의성/활용성: 3051
  품질: 687
  피부타입: 341
  향/냄새: 5905
  흡수력: 1435


MainCategory: 메이크업/뷰티소품
  가격: 3317
  기능/효과: 2768
  디자인: 420
  밀착력/접착력: 1714
  발림성: 3160
  발색력: 2028
  보습력/수분감/쿨링감: 2905
  사용감: 290
  사이즈/두께: 757
  색상: 2742
  성분: 159
  용기: 1114
  용량/개수: 1057
  유통기한: 376
  윤기/피부(톤): 1716

## MainCategory마다 데이터 분할 저장, 일의 자리 수 속성은 제거

In [11]:
skin_care = [item for item in converted_data if item['MainCategory'] == '스킨케어']
hair_body_care = [item for item in converted_data if item['MainCategory'] == '헤어/바디케어']
make_up_beauty = [item for item in converted_data if item['MainCategory'] == '메이크업/뷰티소품']
man_cosmetic = [item for item in converted_data if item['MainCategory'] == '남성화장품']

In [12]:
# MainCategory별로 너무 적은 속성은 데이터에서 제거
# 헤어/바디케어: 그립감, 남성화장품: 커버력, 탄력

filtered_hair_body_care = [data for data in hair_body_care if not any(category == '그립감' for category, _ in data.get('annotation', []))]
filtered_man_cosmetic = [data for data in man_cosmetic if not any((category == '커버력') or (category == '탄력') for category, _ in data.get('annotation', []))]

In [13]:
# MainCategory별로 따로 데이터 저장
file_names = ['스킨케어', '헤어_바디케어', '메이크업_뷰티소품', '남성화장품']
datas = [skin_care, filtered_hair_body_care, make_up_beauty, filtered_man_cosmetic]
for i in range(0, len(file_names)):
    data = datas[i]
    file_name = file_names[i]

    file_path = f'./nlp_data/{file_name}.jsonl'
    with open(file_path, 'w', encoding='utf-8') as json_file:
        for review in data:
            json_line = json.dumps(review, ensure_ascii=False)
            json_file.write(json_line + '\n')